# Apache Kafka Event Producer 3

Submitted by : Mayank Bhardwaj, Neha Jain, Rishupal Singh Chabbra

In the below code, we have created an Event Producer using Kafka Producer. This will act as Producer 3 for the streaming data and will randomly produce the data for hotspot_TERRA_streaming.csv every 10-30 seconds. Additionally, sender_id = "P3" and created_time = current timestamp is appended to the record so as to identify the producer at the consumer end.

#### Topic "climate" is used for the Kafka stream

In [1]:
import sys
import time
import random
import datetime
import csv

hotspot_terra = []
with open('hotspot_TERRA_streaming.csv') as csv_file:
    csv_reader = csv.reader(csv_file,delimiter=',')
    headers = next(csv_reader)    #skipping header
    for row in csv_reader:
        # loading each row as json record and appending to a list so as to send to the consumer
        record = {'latitude': row[0]  , 
                  'longitude': row[1] , 
                  'confidence': row[2] , 
                  'surface_temperature_celcius': row[3],
                  'sender_id':'P3'
            
        }
        
        hotspot_terra.append(record)

In [2]:
# import statements

from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt

    
        
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')   #converting the message to bytes to send
        producer_instance.send(topic_name, key=key_bytes, value=data)   #sending the message
        producer_instance.flush()  # flushing the producer
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:   # exception in case of any issues encounters
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        # connecting to the Kafka server and creating the KafkaProducer object
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'climate'  #Topic for Kafka is climate
    
    print('Publishing records..')
    while True:
        producer = connect_kafka_producer() # connecting to the Kafka and using its object to send messages
        try:
            data = hotspot_terra[random.randrange(len(hotspot_terra)-1)] # random feeding of the messages
            # appending the created_time as current_timestamp
            data['created_time']=str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            # publishing the message as json, converted to bytes before sending
            publish_message(producer, topic, 'jsondata', data)
            # generating the time interval randomly between 10-30 seconds, both inclusive to send the data
            t = random.randint(10,31)
            sleep(t)  # creating an interval of t seconds to send the data
        finally: 
            producer.close()


Publishing records..
Message published successfully. Data: {'confidence': '78', 'sender_id': 'P3', 'latitude': '-36.4005', 'longitude': '144.0756', 'surface_temperature_celcius': '51', 'created_time': '2019-05-28 01:06:03'}
Message published successfully. Data: {'confidence': '63', 'sender_id': 'P3', 'latitude': '-37.0669', 'longitude': '145.3677', 'surface_temperature_celcius': '42', 'created_time': '2019-05-28 01:06:28'}
Message published successfully. Data: {'confidence': '62', 'sender_id': 'P3', 'latitude': '-37.6767', 'longitude': '142.9892', 'surface_temperature_celcius': '41', 'created_time': '2019-05-28 01:06:59'}
Message published successfully. Data: {'confidence': '76', 'sender_id': 'P3', 'latitude': '-37.9199', 'longitude': '145.8475', 'surface_temperature_celcius': '49', 'created_time': '2019-05-28 01:07:20'}


KeyboardInterrupt: 